## Overview

This notebook contains several approaches for representing in a graphical way the journey of a brain image while going to the set neural network architecture. The advantage of a graphical visualization is being able to recognize which features (i.e: edges, pattern, etc) have been captured in each convolutional and pooling layer by the CNN-Model. This l

For this notebook following Data Set is used, containing only 2D images of MRI Segmentation: [Alzheimer's Dataset (4 class of Images)](https://www.kaggle.com/datasets/tourist55/alzheimers-dataset-4-class-of-images)

## Import needed packages and libraries

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Activation, Dense, Flatten, Conv2D, MaxPool2D

import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import random
from sklearn.model_selection import train_test_split
from keras import metrics
import keras.backend as K
import string
from IPython.display import Image, display
#from CAM import get_heatmap
#!pip install keras-visualizer
from keras_visualizer import visualizer
#!pip install graphviz

#!pip install imutils

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, load_img, img_to_array

### Define pfads for both training and validation data set


In [ ]:
# Directory with training pictures
train_dir = os.path.join('/Users/josealbertodiazsalas/Downloads/Alzheimers_Dataset/train')

# Directory with validation pictures
test_dir = os.path.join('/Users/josealbertodiazsalas/Downloads/Alzheimers_Dataset/validation')

In [ ]:

# Define ImageDataGenerator for both train and validation data. Image augmentation can be also set in this step
# All images will be rescaled by 1./255 - Normalization
# Open point: how to deal with unbalanced data

datagen = ImageDataGenerator(rescale = 1./255,
                                  dtype=tf.float32,
                                  validation_split = 0.2,           # setting a validation data set from training data set  
                                  #rotation_range = 90,
                                  #width_shift_range = 0.2,
                                  #height_shift_range = 0.2,
                                  #shear_range = 0.2,
                                  #zoom_range = 0.2,
                                  horizontal_flip = True,
                                  #vertical_flip = True,
                                  #fill_mode='nearest',
                                  #data_format = 'channels_last'
                                  )

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(train_dir,                        # This is the source directory for training images, previously defined
                                                   batch_size = 64,
                                                   target_size = (150, 150),   # All images will be resized to 150x150
                                                   color_mode = 'grayscale',   # RGB, if working with colors
                                                   class_mode = 'sparse',      # for integer labels representation, not hot encoding
                                                   shuffle = True,             # to get the data randomized
                                                   subset = 'training',
                                                   seed = 123                  # if we want to achieved the same results  
                                                   )          
# Flow validation images in batches of 20 using test_datagen generator
validation_generator = datagen.flow_from_directory(train_dir,
                                                   batch_size  = 64,
                                                   target_size = (150, 150),
                                                   color_mode = 'grayscale',
                                                   class_mode  = 'sparse',
                                                   subset = 'validation',
                                                   shuffle = True,
                                                   seed = 123
                                                   )

test_generator = test_datagen.flow_from_directory(test_dir,
                                                   batch_size  = 64,
                                                   target_size = (150, 150),
                                                   color_mode = 'grayscale',
                                                   class_mode  = 'sparse',        
                                                   )        

In [ ]:
print(f'Training Data contains {train_generator.samples} images from type {train_generator.dtype}, following Labels are assigned to the classes: {train_generator.class_indices}')
print('-'*30)
print(f'Validation Data contains {validation_generator.samples} images from type {validation_generator.dtype}, following Labels are assigned to the classes: {validation_generator.class_indices}')

In [ ]:
# Function will plot images in the form of a grid with 1 row x 10 columns
train_images, train_labels = next(train_generator)
validation_images, validation_labels = next(validation_generator)
test_images, test_labels = next(test_generator)

In [ ]:
def plotImages(images_arr):
   fig, axes = plt.subplots(1,20,figsize=(20,20))
   axes = axes.flatten()
   for img, ax in zip(images_arr, axes):
       #print(img.shape)
       ax.imshow(img)
       ax.axis('off')
   plt.tight_layout()
   plt.show()

plotImages(train_images)
print(train_labels)

In [ ]:
#ImageDataGenerator fit() accepts x4 dimensions (number of, width, height, channel)

x=np.concatenate([train_generator.next()[0] for i in range(train_generator.__len__())])
y=np.concatenate([validation_generator.next()[0] for i in range(validation_generator.__len__())])
a=np.concatenate([train_generator.next()[1] for i in range(train_generator.__len__())])
b=np.concatenate([validation_generator.next()[1] for i in range(validation_generator.__len__())])
print(x.shape)
print(y.shape)
print(a.shape)
print(b.shape)

In [ ]:
def convolutional_model():

   model = tf.keras.models.Sequential([
   # Add convolutions and max pooling
   tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 1)),
   tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
   tf.keras.layers.MaxPooling2D(2, 2),

   tf.keras.layers.Conv2D(64, (3,3), activation='relu', name='LastConvLayer'),
   tf.keras.layers.MaxPooling2D(2,2),

   # Add dense layers
   tf.keras.layers.Flatten(),
   tf.keras.layers.Dense(64, activation='relu'),
   tf.keras.layers.Dense(4, activation='softmax'),
   ])

   # Compile the model for multiclass problem
   model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['acc']
                 )
      
   return model


In [ ]:
# Save your untrained model
model = convolutional_model()


# Print the model summary
model.summary()

In [ ]:
# Visualizing Model Architecture
visualizer(model, filename='moderateDem50',format='jpg', view=False)
from PIL import Image
input_path = './moderateDem50.jpg'
im = Image.open(input_path)
im

In [ ]:
# Train your model
print(f'\nMODEL TRAINING:')
history = model.fit(train_generator,      #we might need to defined a Callback as convergence (stopping) criteria
                   epochs=15,
                   verbose = 2,
                   validation_data = validation_generator,
                   #steps_per_epoch = 8      # numbers of images / batch_size?
                   #validation_steps = len(validation_set)/batch_size
                   )   

# Evaluate on the test set
#print(f'\nMODEL EVALUATION:')
#test_loss = model.evaluate_generator(generator=validation_generator)


print(f"Your model was trained for {len(history.epoch)} epochs")

## 1_Visualizing the convolutions and pooling using Keras.layers API

By picking out a few instances of the AD classes it is possible to visualize the journey of an brain image through the convolutions. In this code we also can set the CONVOLUTION_NUMBER variable to look at the output of a specific convolution and thus identify which patterns are being detected.

The Keras API gives us each convolution and each pooling and each dense, etc. as a individual layer. So with the layers API, we can take a look at each layer's outputs.

In [ ]:
# First, let's print out the first X validation labels
print(f'Validation Data set contains following Labels: {validation_generator.class_indices}')
print(validation_labels)

In [ ]:
# Example for x3 images, representing x4 conv layer from left to right
f, axarr = plt.subplots(3,4)
plt.rcParams["figure.figsize"] =(20,9)

# Following images refer to its index within the label array shown above
FIRST_IMAGE=0
SECOND_IMAGE=1
THIRD_IMAGE=2
CONVOLUTION_NUMBER = 24  

# Create a list of each layer's output
layer_outputs = [layer.output for layer in model.layers]

# We can treat each item in the layer as an individual activation model
activation_model = tf.keras.models.Model(inputs = model.input, outputs = layer_outputs)

# By looping through the layers, we can display the journey of the image through the first convolution, first pooling, second convolution, and so on
for x in range(0,4):
 f1 = activation_model.predict(validation_images[FIRST_IMAGE].reshape(1, 150, 150, 1))[x]
 axarr[0,x].imshow(f1[0, : , :, CONVOLUTION_NUMBER], cmap='inferno')
 axarr[0,x].grid(False)
  f2 = activation_model.predict(validation_images[SECOND_IMAGE].reshape(1, 150, 150, 1))[x]
 axarr[1,x].imshow(f2[0, : , :, CONVOLUTION_NUMBER], cmap='viridis')
 axarr[1,x].grid(False)
  f3 = activation_model.predict(validation_images[THIRD_IMAGE].reshape(1, 150, 150, 1))[x]
 axarr[2,x].imshow(f3[0, : , :, CONVOLUTION_NUMBER], cmap='plasma')
 axarr[2,x].grid(False)

## 2_Visualization intermediate representations (hardcoded)

This part includes visualizations of the brain image as it passes through the convolutions and pooling layers. We generate a figure where each row is the output of a layer, and each image in the row is a specific filter in that output feature map. 

In following code, we randomly select a single picture from the training set corresponding to a specific class. By running the code again, we can generate intermediate representations for a variety of training instances.

In [ ]:
# Directory with training pictures
ModerateDemented_dir = os.path.join('/Users/josealbertodiazsalas/Downloads/Alzheimers_Dataset/train/ModerateDemented')
ModerateDemented_names = os.listdir(ModerateDemented_dir)

# Random input image from the training set
ModerateDemented_file = [os.path.join(ModerateDemented_dir, f) for f in ModerateDemented_names]
img_path = random.choice(ModerateDemented_file)

img = load_img(img_path, target_size=(150, 150), color_mode = 'grayscale')
x = img_to_array(img)  # Numpy array with shape (150, 150, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 150, 150, 3)

# Scale by 1/255
x /= 255

# Run the image through the network, thus obtaining all intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)


# These are the names of the layers, so you can have them as part of the plot
layer_names = [layer.name for layer in model.layers[1:]]

# Display the representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
 if len(feature_map.shape) == 4:

   # Just do this for the conv / maxpool layers, not the fully-connected layers
   n_features = feature_map.shape[-1]  # number of features in feature map

   # The feature map has shape (1, size, size, n_features)
   size = feature_map.shape[1]
  
   # Tile the images in this matrix
   display_grid = np.zeros((size, size * n_features))
   for i in range(n_features):
     x = feature_map[0, :, :, i]
     x -= x.mean()
     x /= x.std()
     x *= 64
     x += 128
     x = np.clip(x, 0, 255).astype('uint8')
  
     # Tile each filter into this big horizontal grid
     display_grid[:, i * size : (i + 1) * size] = x
  
   # Display the grid
   scale = 20. / n_features
   plt.figure(figsize=(scale * n_features, scale))
   plt.title(layer_name)
   plt.grid(False)
   plt.imshow(display_grid, aspect='auto', cmap='viridis')

## 3_Keras Grad-CAM class activation visualization

Grad-CAM uses the gradients of any target concept (i.e. classes), flowing into the last convolutional layer of the CNN architecture to produce a coarse localization map highlighting the important regions in the image - captured features - for predicting the concept. The output of Grad-CAM is a heatmap visualization for a given class label. 

Using Grad-CAM and its resulting heatmap, we can visually validate where our network is looking, verifying that it is indeed looking at the correct patterns in the image and activating around those patterns. 

More information to be found on https://keras.io/examples/vision/grad_cam/


In [ ]:
# Directory with training pictures
ModerateDemented_dir = os.path.join('/Users/josealbertodiazsalas/Downloads/Alzheimers_Dataset/train/ModerateDemented')
ModerateDemented_names = os.listdir(ModerateDemented_dir)

# Random input image from the training set
ModerateDemented_file = [os.path.join(ModerateDemented_dir, f) for f in ModerateDemented_names]
img_path = random.choice(ModerateDemented_file)

img = load_img(img_path, target_size=(150, 150), color_mode = 'grayscale')
img_array = img_to_array(img)  # Numpy array with shape (150, 150, 3)
img_array = img_array.reshape((1,) + img_array.shape)  # Numpy array with shape (1, 150, 150, 3)
#x = preprocess_input(x)

'''
def get_img_array(img_path, size):
   img = keras.preprocessing.image.load_img(img_path, target_size=(size))
   array = keras.preprocessing.image.img_to_array(img)
   # We add a dimension to transform our array into a "batch" of size (1, 150, 150, 1)
   array = np.expand_dims(array, axis=0)
   return array
'''

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
   # First, we create a model that maps the input image to the activations of the last conv layer as well as the output predictions
   grad_model = tf.keras.models.Model(
       [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
   )

   # Then, we compute the gradient of the top predicted class for our input image
   # with respect to the activations of the last conv layer
   with tf.GradientTape() as tape:
       last_conv_layer_output, preds = grad_model(img_array)
       if pred_index is None:
           pred_index = tf.argmax(preds[0])
       class_channel = preds[:, pred_index]

   # This is the gradient of the output neuron (top predicted or chosen) with regard to the output feature map of the last conv layer
   grads = tape.gradient(class_channel, last_conv_layer_output)

   # This is a vector where each entry is the mean intensity of the gradient over a specific feature map channel
   pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

   # We multiply each channel in the feature map array by "how important this channel is" with regard to the top predicted class
   # then sum all the channels to obtain the heatmap class activation
   last_conv_layer_output = last_conv_layer_output[0]
   heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
   heatmap = tf.squeeze(heatmap)

   # For visualization purpose, we will also normalize the heatmap between 0 & 1
   heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
   plt.rcParams["figure.figsize"] = (18,8)
   return heatmap.numpy()

last_conv_layer_name = "LastConvLayer"

# Remove last layer's softmax
#model.layers[-1].activation = None

# Print what the top predicted class is
preds = model.predict(img_array)

# Generate class activation heatmap
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

# Display heatmap
plt.rcParams["figure.figsize"] = (18,8)
plt.matshow(heatmap)
#plt.show()

In [ ]:

def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=2):
   # Load the original image
   img = keras.preprocessing.image.load_img(img_path)
   img = keras.preprocessing.image.img_to_array(img)

   # Rescale heatmap to a range 0-255
   heatmap = np.uint8(255 * heatmap)

   # Use jet colormap to colorize heatmap
   jet = plt.cm.get_cmap("jet")

   # Use RGB values of the colormap
   jet_colors = jet(np.arange(256))[:, :3]
   jet_heatmap = jet_colors[heatmap]

   # Create an image with RGB colorized heatmap
   jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
   jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
   jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

   # Superimpose the heatmap on original image
   superimposed_img = jet_heatmap * alpha + img
   superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

   # Save the superimposed image
   #superimposed_img.save(cam_path)

   # Display Grad CAM
   plt.rcParams["figure.figsize"] = (18,8)
   display(Image(cam_path))


display(Image(img_path))
plt.matshow(heatmap)
save_and_display_gradcam(img_path, heatmap)